This notebook will be a recreation of the study `Li, H., Cao, Y., Li, S., Zhao, J., & Sun, Y. (2020). XGBoost model and its application to personal credit evaluation. IEEE Intelligent Systems, 35(3), 52-61.`. The study originally uses dataset from Lending Club, but this source is not available anymore. Though, there copies available on kaggle. The dataset can be found [here](https://www.kaggle.com/datasets/wordsforthewise/lending-club/data).

Authors:
- Adam Janczyszyn
- Hubert Wojewoda
- Jakub Wujec
- Jakub Żmujdzin

### Data Loading

In [2]:
import pandas as pd
df = pd.read_csv('data/credit_card_2018.csv')

C:\Users\jzmuj\AppData\Local\Temp\ipykernel_13164\3524197630.py:2: DtypeWarning: Columns (129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/credit_card_2018.csv')


Let us preprocess the data according to the study's methodology.
We start off by mapping the target variable to 0 and 1. Then, we run sampling for each month - we take 4 times as many good clients as bad clients.

In [3]:
from rr_project.data_wrangling import preprocess_data
preprocessed_df = preprocess_data(df)

C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["issue_d"] = pd.to_datetime(df.issue_d)


Source data shape: (127702, 153). 
Source data target distribution:
target
0    124626
1      3076
Name: count, dtype: int64
Bads for month 1: 420. Goods for month 1: 1680.
Bads for month 2: 323. Goods for month 2: 1292.
Bads for month 3: 390. Goods for month 3: 1560.
Bads for month 4: 394. Goods for month 4: 1576.
Bads for month 5: 361. Goods for month 5: 1444.
Bads for month 6: 310. Goods for month 6: 1240.
Bads for month 7: 271. Goods for month 7: 1084.
Bads for month 8: 213. Goods for month 8: 852.
Bads for month 9: 136. Goods for month 9: 544.
Bads for month 10: 133. Goods for month 10: 532.
Bads for month 11: 80. Goods for month 11: 320.
Bads for month 12: 45. Goods for month 12: 180.
Undersampled data shape: (15380, 153). 
Undersampled data target distribution:
target
0    12304
1     3076
Name: count, dtype: int64


At this first step, we already have discrepancies to the original study:
- though the number of rows is the same, there are 10 additional columns (153 vs 143). They are mainly related to metadata (e.g. url of download) and should not be used in the analysis.
- There is a difference in the number of "bad" clients. The authors used the following mapping: ["Fully Paid", "Current", "In Grace Period"] for good clients, all other for bad clients. Though, the authors obtain over 5k defaults, while we have only a bit over 3k.